# Basic usage of ABCD database

In [ ]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
from abcd import ABCD

First of all, we need to define the url of the database. It could be local or remote:

- direct access: url = 'mongodb://localhost:27017'
- api access: url = 'http://localhost/api'

using with statement to catch the riased exceptions. You may can ignore them  but in that case need to handle all the unexpected events. (cannot connect to db, lost connection, wrong filter, wrong url, etc. )

In [4]:
url = 'mongodb://localhost:27017'
# url = 'http://localhost:5000/api'
abcd =  ABCD(url)

print(abcd)
abcd

MongoDatabase(url=localhost:27017, db=abcd, collection=atoms)


MongoDatabase(url=localhost:27017, db=abcd, collection=atoms)

In [13]:
abcd.print_info()

================== ABCD MongoDB ==================
      type: mongodb
      host: localhost
      port: 27017
        db: abcd
collection: atoms
number of confs: 9707


## Cleanup 

WARNING!! Remove all elements from the database.
Only supported in the case of local access

In [6]:
with abcd as db:
    db.destroy()

## Uploading configurations

In [11]:
from ase.io import iread
from pathlib import Path

direcotry = Path('utils/data/')
file = direcotry / 'bcc_bulk_54_expanded_2_high.xyz'
file = direcotry / 'GAP_6.xyz'

Uploading configurations on-by-one directly from an ase atoms object:

In [16]:
with abcd as db:

    for atoms in iread(file.as_posix(), index=slice(None)):
        
        # Hack to fix the representation of forces
        atoms.calc.results['forces'] = atoms.arrays['force']
        atoms.arrays['force'] = None
            
#         print(atoms)
        db.push(atoms)

Uploading the whole file and injecting to the database on the server side:

In [ ]:
with abcd as db:
    db.upload(file)

In [ ]:
print(abcd.info())

## Query data from the database

The key component is the query string which implementation is based on the GraphQl specification

- query all
- query atoms
- query properties

- histograms/summaries?

In [ ]:
query = {
    'elements': ['Fe', 'H'],
}

with abcd as db:
    traj = [atoms for atoms in db.pull(query)]
    
print(traj)

Query specific properties (like all energies of filetered atoms):

In [ ]:
query = {
    'filter': { 
        elements: ['Fe', 'H'],
    },
    'fields': [
        'energy',
    ]
}

with abcd as db:
    data = db.query(query)
    
print(data)

## Download the whole database

Download the whole database 

In [ ]:
with open('dump.db') as file:
    with abcd as db:
        db.download(file)

## Linking databases

Pull the data from one into another. This function is usefull when you want to build a local database by fetching the data from another repositories.

In [ ]:
with abcd as db:
    db.fetch_from(url='...', query = {})

In [ ]:
print(abcd.info())

## Command line interface

In [ ]:
!abcd --help

In [ ]:
!abcd connect/login 

In [ ]:
!abcd info

In [ ]:
!abcd push --help

In [ ]:
!abcd pull --help

In [ ]:
!abcd query --help

In [ ]:
!abcd download --help

In [ ]:
# search? it is a specific query which returns with the ids only.

# Web interface

flask web server
